In [1]:
import GraphDB.utils as utils
import importlib
import sys, os
from dotenv import load_dotenv

# 모듈 다시 로드
importlib.reload(utils)


#모듈 내 함수 목록 출력 
attributes = dir(utils)
for attribute in attributes:
    print(attribute)


Document
LegalGraphDB
OpenAI
RecursiveCharacterTextSplitter
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__spec__
add_embedding_to_json
article_to_index
ast
calculate_cosine_similarity
cancel_triplet_batch
chunk_split
client
config
contextlib
copy
cosine_similarity
create_refers_to_triplets_list
datetime
dbms
extract_GPT_answer
extract_definition_sentences
extract_document_information
extract_keyword
extract_refers_to_edges
extract_user_keyword
find_clause_pattern
flatten_json
get_batch_answer
get_batch_status
get_gpt_answer
get_refers_to_triplet_json
get_triplet
index_to_article
initialize
json
json_chunk_split
load_dotenv
load_json_as_documents
logging
make_triplet_batch
make_triplet_jsonl
merge_to_json
np
os
pd
process_multiple_pattern
process_range_pattern
re
reranking
search_json
split_clause
split_documents
sys
text_embed
tqdm
traverse_graph_with_hops


In [2]:
## utils에서 불러와서 폴더 내 모든 .json 파일에 대해 refers_to_triplets 생성
##  디렉토리
json_folder_path = "../data/DCM/DCM_json/04/"
print(os.path.isdir(json_folder_path))

utils.get_refers_to_triplet_json(json_folder_path=json_folder_path)

True
## folder path :  ../data/DCM/DCM_json/04/
## folder num :  04
====input_path: ../data/DCM/DCM_json/04//04_regulation_main.json ======
## document_number: 04 law_type: regulation document_type: main
## Number of documents loaded: 6173
## no matches : 4091
## Number of matched patterns: 1050
## 다른 문서, 다른 조의 preceeding_word : 한다)」 ##
## 다른 문서, 다른 조의 preceeding_word : 법률」 ##
## 다른 문서, 다른 조의 preceeding_word : 서명(「전자서명법」 ##
## 다른 문서, 다른 조의 preceeding_word : 법률」 ##
## 다른 문서, 다른 조의 preceeding_word : 한다)」 ##
## 다른 문서, 다른 조의 preceeding_word : 「파생상품시장업무규정」 ##
## 다른 문서, 다른 조의 preceeding_word : 「외국환거래규정」 ##
## 다른 문서, 다른 조의 preceeding_word : 「외국환거래규정」 ##
## 다른 문서, 다른 조의 preceeding_word : 법률」 ##
## 다른 문서, 다른 조의 preceeding_word : 연금저축펀드(「소득세법」 ##
## 다른 문서, 다른 조의 preceeding_word : 규정」 ##
## 다른 문서, 다른 조의 preceeding_word : 「법인세법」 ##
## 다른 문서, 다른 조의 preceeding_word : 법률」 ##
## 다른 문서, 다른 조의 preceeding_word : 「증권의발행및공시등에관한규정」 ##
## 다른 문서, 다른 조의 preceeding_word : 규정」 ##
## 다른 문서, 다른 조의 preceeding_word 

### original codes

In [ ]:
import os

# refers_to_triplets 리스트 초기화
refers_to_triplets = []

# JSON 파일들이 있는 폴더 경로
folder_path = "./data/DCM/DCM_json/02/."

# 폴더 내 모든 .json 파일에 대해 refers_to_triplets 생성
for file_name in os.listdir(folder_path):
    if file_name.endswith(".json"):
        input_path = os.path.join(folder_path, file_name)
        refers_to_triplets.append(utils.create_refers_to_triplets_list(input_path))

print("Finish creating all triplets")


refers_to json 저장 


In [ ]:
#refers_to_triplets를 json 형태로 저장 
output_path = "../data/graph/clause/edge_triplets/02_refers_to_triplets.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(refers_to_triplets, f, ensure_ascii=False, indent=4)

print("Finish saving all triplets")

In [ ]:

CONFIG_PATH = r"C:\Users\Shic\development\GIB\legal_graph\codes\configs\config.json"
config = None
# config_file 가져오기
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)
    print(f"Loaded config data from {CONFIG_PATH}")


# LegalGraphDB 객체 생성
dbms = LegalGraphDB(auradb=False, config=config)



In [ ]:
database = config.get("database") 
data_path = "../data/DCM/DCM_json/01_order_main.json"
node_type = data_path.split("/")[-1].split(".")[0]

# 노드 타입 이름 수정
node_type = f"Clause_{node_type}"

print("=====information of data=====")
print("database : ", database)
print("data_path : ", data_path)
print("node_type : ", node_type)

# data json 파일 불러오기 
with open (data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"{data_path}의 노드 개수 : {len(data)}")

# Clause Graph 구축

In [ ]:

for idx, item in enumerate(data):
    node_id = dbms.create_clause_node(
        node_type=node_type, 
        node_property=item, 
        database=database, 
        embedding=False
    )
    print("idx: ", idx,  "return node_id: ", node_id)
    

# refers_to relation 구축

Create refers_to relation 

In [ ]:
dbms.delete_all_relationship(database=database)

json_path = "../data/graph/clause/edge_triplets/refers_to_triplets.json"
refers_to_triplets = json.load(open(json_path, 'r', encoding='utf-8'))

for triplets in refers_to_triplets:
    for idx, triplet in enumerate(triplets):
        dbms.create_clause_relationship(triplet,database=database)
